[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://github.com/vectara/example-notebooks/blob/main/notebooks/db-query.ipynb)

In [ ]:
import os
import json
import requests
import re

In [2]:
vectara_corpus_id = os.environ['VECTARA_CORPUS_ID']
vectara_customer_id = os.environ['VECTARA_CUSTOMER_ID']
vectara_api_key = os.environ['VECTARA_API_KEY']

def get_post_headers() -> dict:
    """Returns headers that should be attached to each post request."""
    return {
        "x-api-key": vectara_api_key,
        "customer-id": vectara_customer_id,
        "Content-Type": "application/json",
    }

def query(query: str, **kwargs) -> str:

    filter_str = kwargs.get("filter_str", "")
    lambda_val = kwargs.get("lambda_val", 0.0)
    
    corpus_key = {
        "customerId": vectara_customer_id,
        "corpusId": vectara_corpus_id,
        "lexicalInterpolationConfig": {"lambda": lambda_val},
    }
    if len(filter_str) > 0:
        corpus_key["metadataFilter"] = filter_str

    data = {
        "query": [
            {
                "query": query,
                "start": 0,
                "numResults": 10,
                "contextConfig": {
                    "sentencesBefore": 2,
                    "sentencesAfter": 2
                },
                "corpusKey": [corpus_key],
                "summary": [
                    {
                        "responseLang": "eng",
                        "maxSummarizedResults": 5,
                        "summarizerPromptName": "vectara-summary-ext-v1.2.0"
                    },
                ]                    
            }
        ]
    }

    response = requests.post(
        headers=get_post_headers(),
        url="https://api.vectara.io/v1/query",
        data=json.dumps(data),
        timeout=30,
    )

    if response.status_code != 200:
        _logger.error(
            "Query failed %s",
            f"(code {response.status_code}, reason {response.reason}, details "
            f"{response.text})",
        )
        return []

    result = response.json()

    answer = result["responseSet"][0]["summary"][0]["text"]
    return re.sub(r'\[\d+(,\d+){0,5}\]', '', answer)

In [3]:
query("What is the best museum for kids?")

"The best museum for kids in Barcelona is the science museum, according to a visitor's recommendation . It is described as the highlight of their trip for their 6 and 8-year-old children. Additionally, there are other museums worth visiting, such as the Picasso museum and the MACBA . These museums provide excellent experiences for children and families. The Recinte Modernista St Pau is also nearby and highly recommended . Barcelona offers various parks and playgrounds for kids, including one near the beach . Overall, Barcelona provides a range of options for kids to have fun and learn through interactive exhibits and outdoor activities."

In [4]:
query("Which neighborhood has the best Tapas places?")

'The Gracia neighborhood in Barcelona has been recommended as having the best tapas places . Several search results mentioned the presence of great tapas restaurants in the area . It is described as a wonderful neighborhood with easy access to attractions like Sagrada Familia and Parc Guell . The neighborhood is also known for its safety and quietness . The Estrella market nearby offers fresh produce and traditional Catalan food . Overall, Gracia seems to be a highly recommended neighborhood for experiencing the best tapas in Barcelona.'

In [5]:
query("Where can I find night clubs?")

'Night clubs can be found in various locations. One search result mentioned a night club called Club Pipos, located below an apartment, which can be noisy until 5 AM . Another result mentioned a night club near a certain apartment, where noise from lively visitors can be heard until 1-2 AM . Additionally, one search result mentioned a night club named APOLLO, causing potential noise issues until late hours . However, another search result stated that the nightclub across an apartment is only open on Thursdays, Fridays, and Saturdays from midnight to 5:30 AM, with no music heard, but some noise from people coming and going . It is important to note that preferences for noise levels may vary, and it is advisable to consider these factors when choosing a location to visit night clubs.'

In [6]:
query("Where can I find night clubs?", 
      filter_str="doc.date < '2020-01-01'")

'Based on the search results, there are several places where you can find night clubs in Barcelona. One search result mentioned a night club located just two minutes away from a specific location . Another result mentioned a nightclub situated right across an apartment . However, it is worth noting that staying in these areas may result in some noise during the night due to the presence of nightclubs  . Additionally, some search results mentioned that the Eixample district is known for its shops, restaurants, tapas bars, and nightclubs . Overall, it seems that there are various options for finding nightclubs in Barcelona, with some areas being more lively than others.'